# Hyper Parameter Tuning

In [ ]:
# Hyper parameter tuning is done on the preprocessed data sets for selecting the best parameters for a given model
# By changing the combination of parameters in a model, we can try to achieve highest accuracy
# While changing the parameter, we can also continuously variate train-test split in the data set using K Fold cross validation

In [ ]:
# Apply Hyper Parameter tuning on sklearn's Digits Dataset (Classification problem)

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.datasets import load_digits

In [3]:
digits = load_digits()

In [4]:
dir(digits) # Checking the directory of the data set

['DESCR', 'data', 'images', 'target', 'target_names']

In [5]:
inputs = pd.DataFrame(digits.data) # Independent Variables (X)
inputs.head()

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0
1,0.0,0.0,0.0,12.0,13.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,11.0,16.0,10.0,0.0,0.0
2,0.0,0.0,0.0,4.0,15.0,12.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,3.0,11.0,16.0,9.0,0.0
3,0.0,0.0,7.0,15.0,13.0,1.0,0.0,0.0,0.0,8.0,...,9.0,0.0,0.0,0.0,7.0,13.0,13.0,9.0,0.0,0.0
4,0.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,16.0,4.0,0.0,0.0


In [6]:
output = pd.DataFrame(digits.target) # Dependent variable (y)
output.head()

,0
0,0
1,1
2,2
3,3
4,4


In [7]:
output = output.T.squeeze() # converting 'output' dataframe into 'output' series, else warning comes

In [9]:
output.head()

0    0
1    1
2    2
3    3
4    4
Name: 0, dtype: int32

# Choosing best parameters

In [10]:
# Using Train-Test Split method

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(inputs,output,test_size = 0.3)

In [13]:
# Applying Support vector machine (SVM) algorithm
from sklearn import svm

In [14]:
s_model = svm.SVC(kernel = 'rbf',C = 30,gamma = 'auto') # Parameters here are "kernel, C, gamma"

In [15]:
s_model.fit(X_train,y_train)

SVC(C=30, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [16]:
s_model.score(X_test,y_test)

0.39444444444444443

In [17]:
# Using CrossValidation for folding data into different sets of train and test split
from sklearn.model_selection import cross_val_score

In [18]:
cross_val_score(svm.SVC(kernel = 'rbf', C=10, gamma = 'auto'),inputs,output,cv = 5) # cv = 5:-> 5 slices of data in test/train

array([0.46978022, 0.43370166, 0.44289694, 0.5070028 , 0.51267606])

In [19]:
cross_val_score(svm.SVC(kernel = 'linear', C=10, gamma = 'auto'),inputs,output,cv = 5)

array([0.96428571, 0.92265193, 0.96657382, 0.96358543, 0.92957746])

In [20]:
cross_val_score(svm.SVC(kernel = 'rbf', C=1, gamma = 'auto'),inputs,output,cv = 5)

array([0.43406593, 0.40883978, 0.41504178, 0.48739496, 0.49859155])

In [21]:
# We can see different accuracy scores after applying different parameters

In [22]:
# Reworking above process using 'for loop'

In [23]:
kernels = ['linear','rbf']
C = [1,10,20,30]
avg_scores = {}
for kval in kernels:
    for cval in C:
        cv_scores = cross_val_score(svm.SVC(kernel = kval, C=cval, gamma = 'auto'),inputs,output,cv = 5)
        avg_scores[kval + '_'+str(cval)] = np.average(cv_scores)
avg_scores

{'linear_1': 0.9493348726211526,
 'linear_10': 0.9493348726211526,
 'linear_20': 0.9493348726211526,
 'linear_30': 0.9493348726211526,
 'rbf_1': 0.44878680061604637,
 'rbf_10': 0.47321153412608147,
 'rbf_20': 0.47321153412608147,
 'rbf_30': 0.47321153412608147}

In [24]:
# Reworking above process using 'Gridsearch'
from sklearn.model_selection import GridSearchCV

In [25]:
clf = GridSearchCV(svm.SVC(gamma = 'auto'),{
    'C': [1,10,20,30],     #<- This is Parameter Grid, supply parameters for the model here
    'kernel': ['rbf','linear']
},cv = 5,return_train_score = False,iid=False)

clf.fit(inputs,output)
clf.cv_results_ # This is a dataframe, so can be viewed as such

{'mean_fit_time': array([0.42023191, 0.05072761, 0.45919018, 0.0498261 , 0.4221982 ,
        0.04346271, 0.43146586, 0.06730309]),
 'std_fit_time': array([0.01205667, 0.00970971, 0.05444085, 0.00758293, 0.01977676,
        0.00653713, 0.04928719, 0.0049976 ]),
 'mean_score_time': array([0.05168862, 0.01120358, 0.05625496, 0.00960608, 0.05281992,
        0.01327858, 0.05865498, 0.01899705]),
 'std_score_time': array([0.00360434, 0.00391355, 0.00741642, 0.00319912, 0.00640956,
        0.00613782, 0.00845156, 0.00445623]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20, 30, 30],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear',
                    'rbf', 'linear'],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel'

In [26]:
df = pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.420232,0.012057,0.051689,0.003604,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.434066,0.408840,0.415042,0.487395,0.498592,0.448787,0.037208,8
1,0.050728,0.009710,0.011204,0.003914,1,linear,"{'C': 1, 'kernel': 'linear'}",0.964286,0.922652,0.966574,0.963585,0.929577,0.949335,0.019111,1
2,0.459190,0.054441,0.056255,0.007416,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.469780,0.433702,0.442897,0.507003,0.512676,0.473212,0.032221,5
3,0.049826,0.007583,0.009606,0.003199,10,linear,"{'C': 10, 'kernel': 'linear'}",0.964286,0.922652,0.966574,0.963585,0.929577,0.949335,0.019111,1
4,0.422198,0.019777,0.052820,0.006410,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.469780,0.433702,0.442897,0.507003,0.512676,0.473212,0.032221,5
5,0.043463,0.006537,0.013279,0.006138,20,linear,"{'C': 20, 'kernel': 'linear'}",0.964286,0.922652,0.966574,0.963585,0.929577,0.949335,0.019111,1
6,0.431466,0.049287,0.058655,0.008452,30,rbf,"{'C': 30, 'kernel': 'rbf'}",0.469780,0.433702,0.442897,0.507003,0.512676,0.473212,0.032221,5
7,0.067303,0.004998,0.018997,0.004456,30,linear,"{'C': 30, 'kernel': 'linear'}",0.964286,0.922652,0.966574,0.963585,0.929577,0.949335,0.019111,1


In [27]:
df = df[['param_C','param_kernel','mean_test_score']]
df

,param_C,param_kernel,mean_test_score
0,1,rbf,0.448787
1,1,linear,0.949335
2,10,rbf,0.473212
3,10,linear,0.949335
4,20,rbf,0.473212
5,20,linear,0.949335
6,30,rbf,0.473212
7,30,linear,0.949335


In [28]:
dir(clf)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_is_fitted',
 '_estimator_type',
 '_format_results',
 '_get_param_names',
 '_get_tags',
 '_required_parameters',
 '_run_search',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 'classes_',
 'cv',
 'cv_results_',
 'decision_function',
 'error_score',
 'estimator',
 'fit',
 'get_params',
 'iid',
 'inverse_transform',
 'multimetric_',
 'n_jobs',
 'n_splits_',
 'param_grid',
 'pre_dispatch',
 'predict',
 'predict_log_proba',
 'predict_proba',
 'refit',
 'refit_time_',
 'return_train_score',
 'score',
 'scorer_',
 'scoring',
 '

In [29]:
clf.best_score_ # best score of all combination

0.9493348726211526

In [30]:
clf.best_params_ # best parameter for best combination

{'C': 1, 'kernel': 'linear'}

In [31]:
# One shortcoming of GridValue is that it cannot take large number of parameters
# Because of this we use Randomized SearchCV in real time. It randomly combines different parameters for defined no.of iteration

In [32]:
# Reworking above process using 'Randomized SearchCV'
from sklearn.model_selection import RandomizedSearchCV

In [33]:
rs = RandomizedSearchCV(svm.SVC(gamma = 'auto'),{
    'C': [1,10,20,30],
    'kernel': ['rbf','linear']
},cv = 5,return_train_score = False,n_iter = 3, iid=False) # n_iter means no. of combinations, above was 8 

rs.fit(inputs,output)
pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,20,rbf,0.473212
1,20,linear,0.949335
2,10,linear,0.949335


In [34]:
# Because it randomly chooses combinations, it is much faster on very large datasets

# Choosing best model

In [35]:
# Import libraries
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings("ignore")

In [36]:
# Defining parameter Grid
model_params = {
    'svm':{
        'model':svm.SVC(gamma = 'auto'),
        'params':{
            'C':[1,10,20,30],
            'kernel':['rbf','linear','poly','sigmoid']
        }
    },
    'random_forest':{
        'model':RandomForestClassifier(),
        'params':{
            'n_estimators':[1,5,10,50],
            'max_depth':[3,4,5]
        }
    },
    'logistic_regression':{
        'model':LogisticRegression(),
        'params':{
            'C':[1,10,20,30],
            'solver':['lbfgs', 'liblinear']
        }
    },
    'nb_G':{
        'model':GaussianNB(),
        'params':{
            'var_smoothing':[1e-09,1e-08,1e-07,1e-06]
        }
    },
    'nb_M':{
        'model':MultinomialNB(),
        'params':{
            'alpha':[1.0,1.5,1.75,2.0]
        }
    },
    'Decision_tree':{
        'model':DecisionTreeClassifier(),
        'params':{
            'criterion':['gini','entropy'],
            'max_depth':[3,4,5]
        }
    }
}

In [44]:
# Appending the score in a list after fitting the model to each parameter for best result
scores = []
for model_name,mp in model_params.items(): # Iteration of items in the dictionary above
    clf = GridSearchCV(mp['model'],mp['params'],cv = 5,return_train_score = False, iid = False)
    clf.fit(inputs,output)
    scores.append({
        'model':model_name,
        'best':clf.best_score_, # Best score of the model
        'best_param':clf.best_params_  # Best parameter combination of the model
    })

In [45]:
scores

[{'model': 'svm',
  'best': 0.9699354903506467,
  'best_param': {'C': 1, 'kernel': 'poly'}},
 {'model': 'random_forest',
  'best': 0.8942990525469645,
  'best_param': {'max_depth': 5, 'n_estimators': 50}},
 {'model': 'logistic_regression',
  'best': 0.9210543233151656,
  'best_param': {'C': 1, 'solver': 'lbfgs'}},
 {'model': 'nb_G',
  'best': 0.8403522854976886,
  'best_param': {'var_smoothing': 1e-06}},
 {'model': 'nb_M', 'best': 0.8720274056116002, 'best_param': {'alpha': 1.5}},
 {'model': 'Decision_tree',
  'best': 0.7353263986371305,
  'best_param': {'criterion': 'entropy', 'max_depth': 5}}]

In [ ]:
# We can see overall best model is 'SVM' and best parameters are 'C': 1, 'kernel': 'poly'